# Problem1_Classification - Structured_API

In [20]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col, sum
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql import functions as F

## Tạo session

In [21]:
spark = SparkSession.builder.appName("BD_Lab03_1.1").getOrCreate()

## Nhập dữ liệu

In [22]:
path = "/content/creditcard.csv"
data = spark.read.csv(path, header=True, inferSchema=True)

In [23]:
data.show(5)

+----+------------------+-------------------+----------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|              V3|                V4|                 V5|                 V6|                 V7|                V8|                V9|                V10|               V11|               V12|               V13|               V14|               V15|               V16|               V17|                V18|               V19|                V20|                 V21|                V22|     

## Loại bỏ những cột có giá trị bị thiếu và các cột bị trùng lắp

In [24]:
data.count()

284807

In [25]:
data = data.dropna()

In [26]:
data = data.dropDuplicates()

In [27]:
data.count()

283726

## Đếm số lượng mỗi lớp

In [28]:
data.groupBy("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|    1|   473|
|    0|283253|
+-----+------+



Bởi vì số lượng chênh lệch giữa 2 class nên chỉ lấy 500 mẫu class 0.

In [29]:
data_0 = data.filter(col("Class") == 0).orderBy(F.rand(seed=42)).limit(500)
data_1 = data.filter(col("Class") == 1)
data = data_0.union(data_1)
data = data.orderBy(F.rand())

In [30]:
data.groupBy("Class").count().show()

+-----+-----+
|Class|count|
+-----+-----+
|    0|  500|
|    1|  473|
+-----+-----+



## Kết hợp các cột số thành 1 cột vector duy nhất

In [31]:
assembler = VectorAssembler(inputCols=data.columns[:-1], outputCol="raw_features")
data = assembler.transform(data)

## Chuẩn hoá vector

In [32]:
scaler = StandardScaler(inputCol="raw_features", outputCol="features", withMean=True, withStd=True)
scalerModel = scaler.fit(data)
data = scalerModel.transform(data)

data = data.select("features", "Class")

## chia dữ liệu thành 2 phần train, test

In [33]:
train, test = data.randomSplit([0.8, 0.2], seed=42)

## Sử dụng LogisticRegression

In [34]:
lr = LogisticRegression(featuresCol="features", labelCol="Class")
model = lr.fit(train)

## Đánh giá mô hình

In [35]:
summary = model.summary

coefficients = model.coefficients
feature_names = assembler.getInputCols()

print("Coefficients:")
for name, coef in zip(feature_names, coefficients):
    print(f"  {name:<20}: {coef:6f}")

print(f"Intercept: {model.intercept:6f}")

Coefficients:
  Time                : -0.736316
  V1                  : 28.390837
  V2                  : 36.856089
  V3                  : 15.856824
  V4                  : -0.222211
  V5                  : 26.045329
  V6                  : -7.810836
  V7                  : -57.481838
  V8                  : 5.571087
  V9                  : -2.987292
  V10                 : -9.946594
  V11                 : 8.908265
  V12                 : -29.903330
  V13                 : -0.968807
  V14                 : -39.496451
  V15                 : -0.674037
  V16                 : -22.202893
  V17                 : -59.459809
  V18                 : -9.068552
  V19                 : 3.385208
  V20                 : -6.814537
  V21                 : -2.337833
  V22                 : 3.477602
  V23                 : 9.063643
  V24                 : -0.126367
  V25                 : 1.794217
  V26                 : 0.168969
  V27                 : -0.226477
  V28                 : 2.795673
  A

In [36]:
predictions = model.transform(test)

In [37]:
eval_acc = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="accuracy")
eval_precision = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="weightedPrecision")
eval_recall = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName="weightedRecall")
eval_auc = BinaryClassificationEvaluator(labelCol="Class", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

accuracy = eval_acc.evaluate(predictions)
precision = eval_precision.evaluate(predictions)
recall = eval_recall.evaluate(predictions)
auc = eval_auc.evaluate(predictions)


In [38]:
print(f"Accuracy: {accuracy:6f}")
print(f"Precision: {precision:6f}")
print(f"Recall: {recall:6f}")
print(f"AUC: {auc:6f}")

Accuracy: 0.955414
Precision: 0.957220
Recall: 0.955414
AUC: 0.993669
